In [17]:
import pandas as pd
import json
import re

In [ ]:
# Load toolbench bias queries
with open("toolbench_bias_queries.json", "r") as input_file, open("cluster_queries.json", "r") as cluster_queries_file, open("bias_dataset_bfcl_format.jsonl", "w") as output_file:
    # TODO: incorporate return values and api endpoint info into the description
    toolbench_bias_queries = json.load(input_file)
    cluster_queries = json.load(cluster_queries_file)
    
    cluster_id = 0
    types = set()

    clusters = []

    # Decisions for converting to OpenAI format:
    # 1. Function Name will be: f{tool_name}.{api_name} (both converted to snake_case) 
    # 2. Parameters will be "parameters": {"type":"object", "properties": {"{param_name}": {"type": "{type}", "description": "{description}"}}, required: ["{param_name}"]}


    # For Types
    # ENUM, string, STRING --> string
    # DATE (YYYY-MM-DD) --> string
    # BOOLEAN --> boolean
    # NUMBER --> float
    type_mapping = {
        "ENUM": "string",
        "DATE (YYYY-MM-DD)": "string",
        "BOOLEAN": "boolean",
        "NUMBER": "float",
        "STRING": "string",
        "string": "string",
    }

    cluster_id = 1
    for idx in range(0, len(toolbench_bias_queries), 500):
        apis = toolbench_bias_queries[idx]['api_list']

        formatted_tools = []

        
        for api in apis:
            # Convert to snake_case
            api_name = api['api_name'].lower()
            tool_name = api['tool_name'].lower()

            function_name = f"{tool_name}.{api_name}".replace(" ", "_").replace("-", "_").replace("/", "_")

            # remove any non-alphanumeric characters
            function_name = re.sub(r'[^a-zA-Z0-9_\.]', '', function_name)
            

            # Convert to OpenAI format
            required = []
            properties = {}
            for param in api['required_parameters']:
                types.add(param['type'])
                properties[param['name']] = {
                    "type": type_mapping[param['type']],
                    "description": param['description']
                }
                required.append(param['name'])

            for param in api['optional_parameters']:
                types.add(param['type'])
                properties[param['name']] = {
                    "type": type_mapping[param['type']],
                    "description": param['description']
                }

            formatted_tools.append({
                "name": function_name,
                "description": api.get('api_description', ''),
                "parameters": {
                    "type": "object",
                    "properties": properties,
                    "required": required
                }})
        assert cluster_id == cluster_queries[cluster_id - 1]['cluster_id']
        clusters.append({"id": f"bias-{cluster_id}", "function": 
        formatted_tools, "question": [[{"role": "user", "content": query}] for query in cluster_queries[cluster_id - 1]['queries']]})
        cluster_id += 1
    
    print(types)

    # generate jsonl
    for cluster in clusters:
        output_file.write(json.dumps(cluster) + "\n")
    



{'BOOLEAN', 'ENUM', 'STRING', 'DATE (YYYY-MM-DD)', 'NUMBER', 'string'}


In [24]:
# read all names from bias_dataset_bfcl_format.jsonl
with open("bias_dataset_bfcl_format.jsonl", "r") as file:
    names = [func["name"] for line in file for func in json.loads(line)["function"]]
    for name in names:
        print(name)



goapis_geocoding_api.geocding
geocode___forward_and_reverse.forward_geocode
geolocate.get_coordinates
trueway_geocoding.geocode
geokeo_forward_geocoding.search.php
geocode___forward_and_reverse.reverse_geocode
forward__reverse_geocoding.reversegeocoding
geocoding_by_api_ninjas._v1_reversegeocoding
address_from_to_latitude_longitude.coordinates_latitude_longitude_to_address
feroeg___reverse_geocoding.reversegeocode
real_time_news_data.topic_headlines
newscatcher._v1_latest_headlines
theclique.top_headlines
newsdata.news
web_search.newssearch
ip_geolocation_find_ip_location_and_ip_info.get_ip_geolocation
ip_geolocalization_api.ip_address_lookup
free_ip_geolocation.any_ip
ip_address_geolocation.ip_geolocation
ip_geo_location___find_ip_location_and_details.iplocation
domain_whois_lookup_api.domain_name
whois_by_api_ninjas._v1_whois
newly_registered_domains.whois
whois_v2.whois_lookup_v2
whois_.whois
validate_email.validate_email
email_validator._email_validator_validate
email_verifier.veri